In [1]:
import requests
import phenoct
import os
from tqdm import tqdm

try:
    url = "https://projects.pawsey.org.au/appf-quick-data-sharing/XRAYCT_V_X-004481-01_220526112956002_ears_1_200mm_084mu_0645-UniSA-SH0017_220601091114171.rek"
    filename = "test_outputs/example_tube.rek"
    os.mkdir("test_outputs")
    response = requests.get(url, stream=True)
    response.raise_for_status()

    total_size = int(response.headers.get("content-length", 0))

    with open(filename, "wb") as file:
        # Create a progress bar
        progress_bar = tqdm(total=total_size, unit="iB", unit_scale=True)

        for chunk in response.iter_content(chunk_size=1024):
            file.write(chunk)
            progress_bar.update(len(chunk))

        progress_bar.close()

    print(f"File downloaded successfully: {filename}")
except Exception as e:
    print(f"An error occurred: {e}")

tube = phenoct.Tube("test_outputs/example_tube.rek")

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 868M/868M [02:18<00:00, 6.29MiB/s]

File downloaded successfully: test_outputs/example_tube.rek
uint16


In [2]:
tube.write_data_tiff("test_outputs/32.tiff", 32)
# tube.write_data_tiff("16.tiff")

uint16
Min value: 0
Max value: 19690
uint32
Min value: 0
Max value: 1290403840


In [3]:
tube.segment_sample_holder(start_slice=300, stop_slice=2500, debug=False)
# tube.crop_segmented()
tube.write_segmented_data_tiff("test_outputs/seg16.tiff")
tube.write_segmented_data_tiff("test_outputs/seg32.tiff", 32)

Segmenting slice: 2499: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2200/2200 [00:21<00:00, 104.54it/s]


uint16
Min value: 0
Max value: 17507
uint16
Min value: 0
Max value: 17507
uint16
Min value: 0
Max value: 17507
uint32
Min value: 0
Max value: 1147338752


In [4]:
import numpy as np
from PIL import Image

# Load the image using PIL
img = Image.open("test_outputs/seg16.tiff")

# Convert the PIL image to a numpy array
img_arr = np.array(img)

# Print the data type and min/max values
print(f"Data shape: {img_arr.shape}")
print(f"Data type: {img_arr.dtype}")
print(f"Min value: {np.min(img_arr)}")
print(f"Max value: {np.max(img_arr)}")

Data shape: (390, 403)
Data type: uint16
Min value: 0
Max value: 0


In [5]:
# a.crop_segmented()

In [ ]:
# higher attenuation threshold and fewer slices for testing. This is slow.
tube.segment_sample_holder(start_slice=1550, stop_slice=1600, debug=False)

tube.watershed_seeds()

# Uncomment this to open in Napari Viewer.
# tube.view_segmented_data()

tube.write_colourised_tiff("test_outputs/a_cc.tiff")

Segmenting slice: 1599: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:00<00:00, 105.29it/s]


(array([1550, 1550, 1550, ..., 1599, 1599, 1599]), array([100, 100, 100, ..., 191, 191, 191]), array([171, 172, 173, ..., 197, 198, 199]))


In [ ]:
tube.create_animation("test_outputs/tube.mp4")